In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

In [9]:
%%script false
from src import utils
import itertools

num_topics = [i for i in range(4, 74)]
alpha = ['symmetric', 'asymmetric'] + [round(i*0.1, 1) for i in range(1, 10)]
eta = ['symmetric', 'auto'] + [round(i*0.1, 1) for i in range(1, 10)]
passes = [i for i in range(1, 2)]

combinations = list(itertools.product(num_topics, alpha, eta, passes))
combinations_dict = [{'num_topics': item[0], 'alpha': item[1], 'eta': item[2], 'passes': item[3]} for item in combinations]
utils.safe_as_pkl(combinations_dict, '../models/list_parameter_combinations.pkl')

---